# imports

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization, LSTM
from keras.optimizers import SGD

In [2]:
data_path = "Data/"
HRRR_1_path = "Fanova_input/GHI_HRRRonly_Site1.csv"
NAM_1_path = "Fanova_input/GHI_NAMonly_Site1.csv"
SURFRAD_path = "Surfrad/surfrad.csv"

In [3]:
surfrad = pd.read_csv(data_path+SURFRAD_path)
surfrad_1 = surfrad[surfrad["latitude"]==40.05]
surfrad_1 = surfrad_1[surfrad_1["ghi"]<3000]
surfrad_1 = surfrad_1[surfrad_1["ghi"]>-3000]
surfrad_1 = surfrad_1[surfrad_1["#Time"]<=1409346000]
surfrad_1 = surfrad_1[surfrad_1["#Time"]>=1388534400]
surfrad_1.head()

,#Time,latitude,longitude,ghi,ghi_sd,dni,dni_sd,dhi,dhi_sd,zenith_angle,TimeAve
59976,1388534400,40.05,-88.37,-3.44667,0.050742,0.736667,0.066868,0.170000,0.046609,105.07,3600
59977,1388538000,40.05,-88.37,-3.31167,0.141531,0.461667,0.159546,0.181667,0.053652,116.23,3600
59978,1388541600,40.05,-88.37,-3.33500,0.157119,1.901670,0.734268,0.233333,0.050979,127.66,3600
59979,1388545200,40.05,-88.37,-3.31333,0.164128,0.850000,1.129050,0.275000,0.047389,139.07,3600
59980,1388548800,40.05,-88.37,-3.24167,0.136905,0.716667,0.388027,0.283333,0.037582,149.96,3600


In [4]:
HRRR_1 = pd.read_csv(data_path+HRRR_1_path)
HRRR_1

,time,GHI_Meas,HRRR_GHI_inline,HRRR_DNI,HRRR_DHI,HRRR_CloudWaterCal,HRRR_CloudIceCal,HRRR_Temp2m,HRRR_WindSpeed_u_10m,HRRR_WindSpeed_v_10m,...,HRRR_HCDC,HRRR_MCDC,HRRR_LCDC,HRRR_CldTop,HRRR_CldBtm,ClrSkyGHI,ClrSkyDNI,ClrSkyDHI,ClrSkyZenith,ClrSkyAzimuth
0,1409346000,540.391663,607.083000,768.254456,0.000000,0.000823,0.000000e+00,304.175000,-0.198077,5.208660,...,0.000000,0.000000,3.000010,76.302300,52.893400,634.460022,870.124451,91.879997,51.444038,246.143753
1,1409342400,661.109985,744.190308,720.544312,220.050003,0.000823,0.000000e+00,304.174408,-0.198077,5.208647,...,0.000000,0.000000,3.000000,76.302078,52.893230,777.750000,909.281860,98.019997,41.636086,230.762222
2,1409338800,792.658325,537.634766,877.587646,130.429993,0.000000,0.000000e+00,303.096039,0.432659,5.003021,...,0.000000,0.250000,4.753844,0.000000,0.000000,870.809998,929.749878,101.339996,34.156975,209.788528
3,1409335200,644.031677,864.191589,0.000000,353.760010,0.105219,0.000000e+00,303.053711,-0.448537,3.945045,...,0.000000,0.000000,17.685680,2081.108398,1494.297485,906.400024,936.634583,102.500000,30.885010,182.710983
4,1409331600,758.618347,834.612244,583.694153,358.500000,0.002153,0.000000e+00,301.721436,-0.776505,3.536844,...,0.000000,0.000000,1.625000,292.447418,203.702408,881.799988,931.458191,101.699997,33.134167,154.883499
5,1409328000,385.956665,746.927979,853.340942,128.130005,0.000000,0.000000e+00,299.616150,-2.015805,3.793525,...,0.000000,2.627245,3.489520,0.000000,0.000000,798.869995,913.119019,98.820000,39.957813,132.636551
6,1409324400,82.561668,60.782928,0.000000,262.109985,0.083588,0.000000e+00,296.646301,-0.504914,2.266212,...,0.000000,30.676458,29.794535,4014.807861,2692.199707,664.059998,877.234558,93.279999,49.426701,116.345238
7,1409320800,86.793335,110.865341,0.000000,73.349998,0.270241,1.000000e-06,296.346497,-2.156847,3.637644,...,0.000000,51.129318,47.851612,5521.942383,2274.385498,488.179993,812.955933,83.779999,60.196224,103.856354
8,1409317200,68.076668,73.049545,0.000000,84.260002,0.094809,0.000000e+00,295.728516,-2.212134,2.210829,...,0.000000,80.020958,5.420216,5521.974609,4237.700195,287.220001,691.250488,67.669998,71.529182,93.384567
9,1409313600,41.453335,57.340107,0.000000,44.419998,0.022602,0.000000e+00,294.589813,-1.806311,0.994536,...,0.000000,51.134716,1.361357,5520.698731,4239.943848,88.690002,407.417480,38.090000,82.987221,83.751534


In [29]:
NAM_1 = pd.read_csv(data_path+NAM_1_path)
NAM_1

,time,GHI_Meas,NAM_Total_Sh_Dw_inline,NAM_DNI,NAM_DHI,NAM_CloudWaterCal,NAM_CloudIceCal,NAM_Temp2m,NAM_WindSpeed_u_10m,NAM_WindSpeed_v_10m,...,NAM_HCDC,NAM_MCDC,NAM_LCDC,NAM_CldTop,NAM_CldBtm,ClrSkyGHI,ClrSkyDNI,ClrSkyDHI,ClrSkyZenith,ClrSkyAzimuth
0,1419804000,58.886665,44.891376,0.114000,38.660000,0.000000,0.008507,272.995239,1.702605,-0.765027,...,62.281029,0.000000,0.000000,9393.158203,9360.922852,56.660000,307.458405,29.690001,85.130486,233.909927
1,1419800400,196.776672,138.501266,0.000000,93.199997,0.000000,0.040647,273.839569,2.480774,-1.277663,...,100.000000,0.000000,0.000000,10627.422852,8728.806641,211.399994,646.578613,60.169998,76.538765,222.996597
2,1419796800,236.273331,204.326904,0.000000,133.679993,0.000000,0.063180,273.946716,2.453802,-1.525390,...,100.000000,0.000000,0.000000,10639.662109,7565.882324,346.059998,778.312805,75.019997,69.663437,210.514282
3,1419793200,299.523346,390.029907,26.169886,280.649994,0.000000,0.017755,273.712891,2.852232,-1.601933,...,55.185085,0.000000,0.000000,10171.539062,9175.554688,435.609985,837.128174,82.459999,65.082542,196.433594
4,1419789600,243.921661,272.028137,0.022242,199.210007,0.000000,0.060335,273.051636,3.034476,-1.799952,...,100.000000,89.907753,0.000000,10654.740234,7035.002930,469.859985,856.103455,84.949997,63.314240,181.255127
5,1419786000,201.128326,314.084961,0.023281,192.809998,0.000000,0.056079,272.960052,2.840981,-1.855275,...,100.000000,86.967087,0.153972,10125.111328,7551.460938,445.420013,843.318909,83.190002,64.596123,165.985413
6,1419782400,153.339996,168.002029,0.000000,145.330002,0.000000,0.060062,272.114471,2.660431,-1.434123,...,100.000000,100.000000,6.168488,10673.732422,6659.205566,364.679993,793.226624,76.690002,68.753624,151.678604
7,1419778800,103.165001,140.043701,0.000000,106.919998,0.000000,0.038775,271.602753,2.643854,-2.112575,...,100.000000,99.169525,99.562523,10200.061523,678.745605,236.490005,679.349792,63.400002,75.298576,138.930145
8,1419775200,40.750000,46.059433,0.000000,27.320000,0.019966,0.046909,271.160370,2.983401,-2.920613,...,100.000000,100.000000,100.000000,10677.883789,549.703857,81.779999,398.510132,36.799999,83.650063,127.783173
9,1419771600,-1.788333,0.000000,0.000000,0.000000,0.032582,0.035891,271.048706,2.984208,-2.554289,...,95.615974,3.437192,100.000000,10092.758789,482.852509,0.000000,0.000000,0.000000,93.291435,117.937126


In [30]:
hrrr_time = HRRR_1['time']
hrrr_ghi_meas = HRRR_1[' GHI_Meas']
hrrr_ghi_inline = HRRR_1[' HRRR_GHI_inline']

nam_time = NAM_1['time']
nam_ghi_meas = NAM_1[' GHI_Meas']
nam_ghi_inline = NAM_1[' NAM_Total_Sh_Dw_inline']

surfrad_time = surfrad_1['#Time']
surfrad_ghi = surfrad_1['ghi']

In [31]:
plt.clf()
fig, ax = plt.subplots()
fig.set_size_inches(100, 3)
#ax.axis('equal')
ax.plot(hrrr_time,hrrr_ghi_meas,alpha = 0.5, color = "r")
ax.plot(hrrr_time,hrrr_ghi_inline,alpha = 0.5, color = "b")
ax.plot(nam_time,nam_ghi_inline,alpha = 0.5, color = "y")
# ax.plot(surfrad_time,surfrad_ghi,alpha = 0.5)
#plt.show()
plt.savefig("GHI_HRRR.pdf")

In [32]:
# check data lap
plt.clf()
fig, ax = plt.subplots()
fig.set_size_inches(100, 3)
#ax.axis('equal')
ax.scatter(hrrr_time,np.ones(hrrr_time.shape[0]),alpha = 0.5, color = "r", s = 0.1)
#ax.plot(hrrr_time,hrrr_ghi_inline,alpha = 0.5, color = "b")
ax.scatter(nam_time,np.ones(nam_time.shape[0]),alpha = 0.5, color = "b", s = 0.1)
# ax.plot(surfrad_time,surfrad_ghi,alpha = 0.5)
#plt.show()
plt.savefig("time_difference.pdf")

In [33]:
print(hrrr_time.shape)
print(nam_time.shape)
np.intersect1d(nam_time, hrrr_time).shape

(3144,)
(5613,)


(2256,)

# blend on only one model
technically this can't be called as model blending huh

# data processing
* input NAM data
* observe data(hist)
* normalize?
* split data and label
* split into train, val and test data
* further processing (zeros?)


# observe and analysis data

In [34]:
nam_array = np.array(NAM_1)
print("data shape:")
print(nam_array.shape)
print("data max:")
print(np.max(nam_array,axis=0))
print("data min:")
print(np.min(nam_array,axis=0))
# check data distribution
plt.clf()
fig, ax = plt.subplots()
# ax.hist(nam_array[:,1])
for i in range(nam_array.shape[1]-1):
    ax = plt.subplot(5,5,i+1)
    ax.hist(nam_array[:,i+1],"auto")
    ax.set_title(NAM_1.columns[i+1])
fig.set_size_inches(50, 50)
# ax.axis('equal')
# plt.show()
plt.savefig("hists")

data shape:
(5613, 26)
data max:
[  1.41980400e+09   1.09207666e+03   1.01643518e+03   1.13439844e+03
   7.87570007e+02   3.41079092e+00   4.89118010e-01   3.06345245e+02
   1.17577190e+01   1.32357864e+01   1.01131516e+05   1.00000000e+02
   6.95000000e+01   1.00000000e+00   1.00000000e+00   1.00000000e+02
   1.00000000e+02   1.00000000e+02   1.00000000e+02   1.54209609e+04
   1.37756279e+04   1.00588000e+03   9.69347290e+02   1.04720001e+02
   1.63301285e+02   3.59997711e+02]
data min:
[  1.38853440e+09  -5.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   2.49341904e+02
  -1.01259031e+01  -1.18527613e+01   9.61956016e+04   1.00000000e+00
   1.60000000e+01   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00  -9.52302856e+01
  -8.72248993e+01   0.00000000e+00   0.00000000e+00   0.00000000e+00
   1.66446590e+01   1.84799999e-03]


from the plots we know that:
* some minus in ground truth
* every column have A LOT OF zeros
* scale differs
* not all positive data(have minus data for directions)

possible approaches:
* extra dimension for zeros
* add small number (e^-30)

## split training and testing
data have time meaning:
1. put time as one dimension then shuffle data
1. using LSTM
1. using Markov (time series cnn?)(look at the paper)
***
## for testing:
1. just focus on one site and use this
1. use one site to train, another site to test
1. concantante all the stations and one extra one-hot dimension for location, or put L&A in it


## approach 1: shuffle

In [35]:
# shuffle
rand_nam_1 = nam_array
print(rand_nam_1.shape)
#np.random.shuffle(rand_nam_1)
# split
train_size = 5000
valid_size = 300
rand_nam_1_train = rand_nam_1[:train_size]
#np.random.shuffle(rand_nam_1_train)
rand_nam_1_valid = rand_nam_1[train_size:train_size+valid_size]
rand_nam_1_test = rand_nam_1[train_size+valid_size:]
print(rand_nam_1_train.shape)
print(rand_nam_1_valid.shape)
print(rand_nam_1_test.shape)

# label
rand_nam_1_train_label = rand_nam_1_train[:,1]
rand_nam_1_valid_label = rand_nam_1_valid[:,1]
rand_nam_1_test_label = rand_nam_1_test[:,1]

mask = np.ones(rand_nam_1.shape[1],dtype = np.bool)
mask[1] = 0
print(mask)
rand_nam_1_train_data = rand_nam_1_train[:,mask]
rand_nam_1_valid_data = rand_nam_1_valid[:,mask]
rand_nam_1_test_data = rand_nam_1_test[:,mask]

print(rand_nam_1_train_data.shape)

(5613, 26)
(5000, 26)
(300, 26)
(313, 26)
[ True False  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True]
(5000, 25)


In [41]:
## training
nn = Sequential()
#nn.add()
nn.add(BatchNormalization(input_shape = rand_nam_1_train_data[0].shape))
nn.add(Dense(500,activation = 'relu'))
nn.add(Dense(50,activation = 'relu'))
#nn.add(Dense(50,activation = 'relu'))
nn.add(Dense(200, activation = 'relu'))
nn.add(Dense(1, activation = 'relu'))
nn.compile(optimizer='rmsprop', loss='mean_squared_error')
# sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
# nn.compile(optimizer = sgd,loss = 'mean_squared_error')
nn.fit(rand_nam_1_train_data, rand_nam_1_train_label, epochs = 50, batch_size = 50)


Epoch 1/50
5000/5000 [==============================] - 1s 149us/step - loss: 30751.9126
Epoch 2/50
5000/5000 [==============================] - 0s 44us/step - loss: 13752.7609
Epoch 3/50
5000/5000 [==============================] - 0s 44us/step - loss: 12376.7992
Epoch 4/50
5000/5000 [==============================] - 0s 44us/step - loss: 11964.0668
Epoch 5/50
5000/5000 [==============================] - 0s 50us/step - loss: 11933.4424
Epoch 6/50
5000/5000 [==============================] - 0s 50us/step - loss: 11417.8889
Epoch 7/50
5000/5000 [==============================] - 0s 54us/step - loss: 12166.9523
Epoch 8/50
5000/5000 [==============================] - 0s 53us/step - loss: 11954.2345
Epoch 9/50
5000/5000 [==============================] - 0s 65us/step - loss: 11566.3095
Epoch 10/50
5000/5000 [==============================] - 0s 61us/step - loss: 11242.9942
Epoch 11/50
5000/5000 [==============================] - 0s 52us/step - loss: 11497.1166
Epoch 12/50
3100/5000 [======

KeyboardInterrupt: 

In [67]:
rand_nam_1_train_data = rand_nam_1_train_data.reshape((rand_nam_1_train_data.shape[0],1,rand_nam_1_train_data.shape[1]))
rand_nam_1_valid_data = rand_nam_1_valid_data.reshape((rand_nam_1_valid_data.shape[0],1,rand_nam_1_valid_data.shape[1]))

In [55]:
## training LSTM
ls = Sequential()
#nn.add()
ls.add(BatchNormalization(input_shape = rand_nam_1_train_data.shape[1:]))
ls.add(LSTM(200, input_shape = rand_nam_1_train_data.shape))
#nn.add(Dense(50,activation = 'relu'))
#nn.add(Dense(50,activation = 'relu'))
#nn.add(Dense(200, activation = 'relu'))
ls.add(Dense(1, activation = 'relu'))
ls.compile(optimizer='adam', loss='mae')
# sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
# nn.compile(optimizer = sgd,loss = 'mean_squared_error')
ls.fit(rand_nam_1_train_data, rand_nam_1_train_label, epochs = 500, batch_size = 72)



Epoch 1/500
5000/5000 [==============================] - 2s 326us/step - loss: 173.0666
Epoch 2/500
5000/5000 [==============================] - 0s 94us/step - loss: 165.4777
Epoch 3/500
5000/5000 [==============================] - 0s 97us/step - loss: 155.4059
Epoch 4/500
5000/5000 [==============================] - 0s 91us/step - loss: 147.8194
Epoch 5/500
5000/5000 [==============================] - 0s 93us/step - loss: 142.5984
Epoch 6/500
5000/5000 [==============================] - 1s 115us/step - loss: 138.5427
Epoch 7/500
5000/5000 [==============================] - 1s 104us/step - loss: 135.1132
Epoch 8/500
5000/5000 [==============================] - 1s 107us/step - loss: 131.9063
Epoch 9/500
5000/5000 [==============================] - 1s 104us/step - loss: 129.2175
Epoch 10/500
5000/5000 [==============================] - 1s 105us/step - loss: 126.4769
Epoch 11/500
5000/5000 [==============================] - 0s 98us/step - loss: 123.9966
Epoch 12/500
5000/5000 [===========

5000/5000 [==============================] - 0s 98us/step - loss: 55.3096
Epoch 95/500
5000/5000 [==============================] - 0s 93us/step - loss: 55.1556
Epoch 96/500
5000/5000 [==============================] - 0s 93us/step - loss: 53.6708
Epoch 97/500
5000/5000 [==============================] - 0s 92us/step - loss: 55.1310
Epoch 98/500
5000/5000 [==============================] - 0s 91us/step - loss: 55.0066
Epoch 99/500
5000/5000 [==============================] - 0s 99us/step - loss: 53.8815
Epoch 100/500
5000/5000 [==============================] - 1s 149us/step - loss: 54.5667
Epoch 101/500
5000/5000 [==============================] - 0s 96us/step - loss: 54.9509
Epoch 102/500
5000/5000 [==============================] - 0s 99us/step - loss: 53.8010
Epoch 103/500
5000/5000 [==============================] - 1s 103us/step - loss: 53.5913
Epoch 104/500
5000/5000 [==============================] - 0s 97us/step - loss: 54.4009
Epoch 105/500
5000/5000 [========================

5000/5000 [==============================] - 0s 95us/step - loss: 46.3468
Epoch 188/500
5000/5000 [==============================] - 0s 96us/step - loss: 46.9617
Epoch 189/500
5000/5000 [==============================] - 0s 95us/step - loss: 45.9037
Epoch 190/500
5000/5000 [==============================] - 0s 96us/step - loss: 45.9630
Epoch 191/500
5000/5000 [==============================] - 0s 95us/step - loss: 47.0567
Epoch 192/500
5000/5000 [==============================] - 1s 109us/step - loss: 45.8629
Epoch 193/500
5000/5000 [==============================] - 0s 94us/step - loss: 46.2454
Epoch 194/500
5000/5000 [==============================] - 0s 94us/step - loss: 45.9075
Epoch 195/500
5000/5000 [==============================] - 0s 95us/step - loss: 46.3052
Epoch 196/500
5000/5000 [==============================] - 0s 94us/step - loss: 45.5409
Epoch 197/500
5000/5000 [==============================] - 0s 95us/step - loss: 47.0439
Epoch 198/500
5000/5000 [====================

5000/5000 [==============================] - 0s 95us/step - loss: 42.9235
Epoch 280/500
5000/5000 [==============================] - 0s 94us/step - loss: 42.4889
Epoch 281/500
5000/5000 [==============================] - 0s 95us/step - loss: 41.9515
Epoch 282/500
5000/5000 [==============================] - 0s 95us/step - loss: 42.0754
Epoch 283/500
5000/5000 [==============================] - 1s 101us/step - loss: 42.2142
Epoch 284/500
5000/5000 [==============================] - 1s 103us/step - loss: 41.7986
Epoch 285/500
5000/5000 [==============================] - 0s 95us/step - loss: 42.0946
Epoch 286/500
5000/5000 [==============================] - ETA: 0s - loss: 41.43 - 0s 95us/step - loss: 41.6503
Epoch 287/500
5000/5000 [==============================] - 0s 95us/step - loss: 41.7860
Epoch 288/500
5000/5000 [==============================] - 0s 96us/step - loss: 41.0625
Epoch 289/500
5000/5000 [==============================] - 0s 95us/step - loss: 40.8567
Epoch 290/500
5000/5

5000/5000 [==============================] - 1s 119us/step - loss: 38.2170
Epoch 372/500
5000/5000 [==============================] - 1s 117us/step - loss: 38.8887
Epoch 373/500
5000/5000 [==============================] - 1s 118us/step - loss: 39.0474
Epoch 374/500
5000/5000 [==============================] - 1s 142us/step - loss: 38.0217
Epoch 375/500
5000/5000 [==============================] - 1s 102us/step - loss: 38.2203
Epoch 376/500
5000/5000 [==============================] - 1s 105us/step - loss: 38.3210
Epoch 377/500
5000/5000 [==============================] - 0s 99us/step - loss: 37.6903
Epoch 378/500
5000/5000 [==============================] - 1s 105us/step - loss: 38.5918
Epoch 379/500
5000/5000 [==============================] - 0s 99us/step - loss: 38.8030
Epoch 380/500
5000/5000 [==============================] - 1s 102us/step - loss: 39.6668
Epoch 381/500
5000/5000 [==============================] - 1s 105us/step - loss: 39.0183
Epoch 382/500
5000/5000 [============

5000/5000 [==============================] - 0s 98us/step - loss: 35.0430
Epoch 464/500
5000/5000 [==============================] - 0s 96us/step - loss: 35.8244
Epoch 465/500
5000/5000 [==============================] - 0s 97us/step - loss: 35.5086
Epoch 466/500
5000/5000 [==============================] - 0s 97us/step - loss: 35.3544
Epoch 467/500
5000/5000 [==============================] - 0s 96us/step - loss: 35.2388
Epoch 468/500
5000/5000 [==============================] - 0s 97us/step - loss: 35.3770
Epoch 469/500
5000/5000 [==============================] - 0s 96us/step - loss: 35.6363
Epoch 470/500
5000/5000 [==============================] - 0s 97us/step - loss: 34.8831
Epoch 471/500
5000/5000 [==============================] - 0s 96us/step - loss: 35.4411
Epoch 472/500
5000/5000 [==============================] - 0s 95us/step - loss: 35.8935
Epoch 473/500
5000/5000 [==============================] - 0s 96us/step - loss: 34.5927
Epoch 474/500
5000/5000 [=====================

In [56]:
pred = (ls.predict(rand_nam_1_valid_data))

In [57]:
# print(pred)
# rand_nam_1_valid_label = (rand_nam_1_valid_label+np.abs(rand_nam_1_valid_label))/2.
print(np.linalg.norm(rand_nam_1_valid_label-pred.reshape(pred.shape[0]),2))
print(np.linalg.norm(rand_nam_1_valid_label-rand_nam_1_valid_data[:,1],2))

1219.94073396
1491.77426134


In [59]:
# visualize results
plt.clf()
fig.clear()
fig, ax = plt.subplots()
fig.set_size_inches(100, 3)

ax.plot(rand_nam_1_valid_data[:,0],rand_nam_1_valid_label.reshape(rand_nam_1_valid_label.shape[0]),alpha = 0.5, color = "k")
ax.plot(rand_nam_1_valid_data[:,0],rand_nam_1_valid_data[:,1],alpha = 0.5, color = "y")
ax.plot(rand_nam_1_valid_data[:,0],pred,alpha = 0.5, color = "g")

#plt.show()
plt.savefig("model_2.pdf")

the 1 site, 1 model data bring the data down to nearly half

## concatnate all sites
concatnate all 7 sites of nam, adding L&A into the data

In [60]:
NAM_1_path = "Fanova_input/GHI_NAMonly_Site1.csv"
NAM_1 = pd.read_csv(data_path+NAM_1_path)
NAM_1['Latitude'] = 40.05
NAM_1['Longitude'] = -88.37
print(NAM_1.shape)

NAM_2_path = "Fanova_input/GHI_NAMonly_Site2.csv"
NAM_2 = pd.read_csv(data_path+NAM_2_path)
NAM_2['Latitude'] = 40.13
NAM_2['Longitude'] = -105.24
print(NAM_2.shape)

NAM_3_path = "Fanova_input/GHI_NAMonly_Site3.csv"
NAM_3 = pd.read_csv(data_path+NAM_3_path)
NAM_3['Latitude'] = 36.63
NAM_3['Longitude'] = -116.02
print(NAM_3.shape)

NAM_4_path = "Fanova_input/GHI_NAMonly_Site4.csv"
NAM_4 = pd.read_csv(data_path+NAM_4_path)
NAM_4['Latitude'] = 48.31
NAM_4['Longitude'] = -105.1
print(NAM_4.shape)

NAM_5_path = "Fanova_input/GHI_NAMonly_Site5.csv"
NAM_5 = pd.read_csv(data_path+NAM_5_path)
NAM_5['Latitude'] = 34.25
NAM_5['Longitude'] = -89.87
print(NAM_5.shape)

NAM_6_path = "Fanova_input/GHI_NAMonly_Site6.csv"
NAM_6 = pd.read_csv(data_path+NAM_6_path)
NAM_6['Latitude'] = 40.72
NAM_6['Longitude'] = -77.93
print(NAM_6.shape)

NAM_7_path = "Fanova_input/GHI_NAMonly_Site7.csv"
NAM_7 = pd.read_csv(data_path+NAM_7_path)
NAM_7['Latitude'] = 43.73
NAM_7['Longitude'] = -96.62
print(NAM_7.shape)

nam_frames = [NAM_1, NAM_2, NAM_3, NAM_4, NAM_5, NAM_6, NAM_7]
NAM_ALL = pd.concat(nam_frames)
print(NAM_ALL.shape)
NAM_ALL_array = np.array(NAM_ALL)

(5613, 28)
(4165, 28)
(2012, 28)
(4366, 28)
(4423, 28)
(5901, 28)
(5079, 28)
(31559, 28)


## train on 1 model, 7 sites
try using 6 sites to train, 1 site to test

* note from siyuan: first split then shuffle,
  * use LSTM

## LSTM approach

## data preparation

In [80]:
# shuffle
rand_nam_all = NAM_ALL_array
print(rand_nam_all.shape)
# np.random.shuffle(rand_nam_1)
# split
train_size = 31559-5079
valid_size = 3000
rand_nam_all_train = rand_nam_all[:train_size]
rand_nam_all_valid = rand_nam_all[train_size:train_size+valid_size]
rand_nam_all_test = rand_nam_all[train_size+valid_size:]
print(rand_nam_all_train.shape)
print(rand_nam_all_valid.shape)
print(rand_nam_all_test.shape)

# label
rand_nam_all_train_label = rand_nam_all_train[:,1]
rand_nam_all_valid_label = rand_nam_all_valid[:,1]
rand_nam_all_test_label = rand_nam_all_test[:,1]

mask = np.ones(rand_nam_all.shape[1],dtype = np.bool)
mask[1] = 0
print(mask)
rand_nam_all_train_data = rand_nam_all_train[:,mask]
rand_nam_all_valid_data = rand_nam_all_valid[:,mask]
rand_nam_all_test_data = rand_nam_all_test[:,mask]

print(rand_nam_all_train_data.shape)

(31559, 28)
(26480, 28)
(3000, 28)
(2079, 28)
[ True False  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]
(26480, 27)


In [81]:
rand_nam_all_train_data = rand_nam_all_train_data.reshape((rand_nam_all_train_data.shape[0],1,rand_nam_all_train_data.shape[1]))
rand_nam_all_valid_pred = rand_nam_all_valid_data[:,1]
rand_nam_all_valid_data = rand_nam_all_valid_data.reshape((rand_nam_all_valid_data.shape[0],1,rand_nam_all_valid_data.shape[1]))

In [97]:
## training LSTM
ls = Sequential()
#nn.add()
ls.add(BatchNormalization(input_shape = rand_nam_all_train_data.shape[1:]))
ls.add(LSTM(100, input_shape = rand_nam_all_train_data.shape))
#nn.add(Dense(50,activation = 'relu'))
#nn.add(Dense(50,activation = 'relu'))
nn.add(Dense(200, activation = 'relu'))
ls.add(Dense(1, activation = 'relu'))
ls.compile(optimizer='adam', loss='mae')
# sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
# nn.compile(optimizer = sgd,loss = 'mean_squared_error')
ls.fit(rand_nam_all_train_data, rand_nam_all_train_label, epochs = 50, batch_size = 72)

Epoch 1/50
26480/26480 [==============================] - 4s 138us/step - loss: 157.8084
Epoch 2/50
26480/26480 [==============================] - 2s 71us/step - loss: 141.0602
Epoch 3/50
26480/26480 [==============================] - 2s 62us/step - loss: 131.3926
Epoch 4/50
26480/26480 [==============================] - 2s 60us/step - loss: 123.2487
Epoch 5/50
26480/26480 [==============================] - 2s 59us/step - loss: 116.3711
Epoch 6/50
26480/26480 [==============================] - 2s 58us/step - loss: 110.4942
Epoch 7/50
26480/26480 [==============================] - 2s 59us/step - loss: 105.3094
Epoch 8/50
26480/26480 [==============================] - 2s 60us/step - loss: 100.5747
Epoch 9/50
26480/26480 [==============================] - 2s 57us/step - loss: 96.3646
Epoch 10/50
26480/26480 [==============================] - 1s 57us/step - loss: 92.3583
Epoch 11/50
26480/26480 [==============================] - 2s 57us/step - loss: 88.8886
Epoch 12/50
26480/26480 [=======

In [100]:
pred = (ls.predict(rand_nam_all_valid_data))

In [101]:
# print(pred)
# rand_nam_1_valid_label = (rand_nam_1_valid_label+np.abs(rand_nam_1_valid_label))/2.
print(np.linalg.norm(rand_nam_all_valid_label-pred.reshape(pred.shape[0]),2))
print(np.linalg.norm(rand_nam_all_valid_label-rand_nam_all_valid_pred,2))

# visualize results
plt.clf()
fig.clear()
fig, ax = plt.subplots()
fig.set_size_inches(500, 3)

ax.plot(rand_nam_all_valid_data[:,0,0],rand_nam_all_valid_label.reshape(rand_nam_all_valid_label.shape[0]),alpha = 0.5, color = "k")
ax.plot(rand_nam_all_valid_data[:,0,0],rand_nam_all_valid_pred,alpha = 0.5, color = "y")
ax.plot(rand_nam_all_valid_data[:,0,0],pred,alpha = 0.5, color = "g")

#plt.show()
plt.savefig("model_3.pdf")

6253.69239809
7795.35078093


/usr/local/lib/python3.6/site-packages/matplotlib/pyplot.py:523: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [88]:
rand_nam_all_valid_data[:,0,0]

array([  1.41997680e+09,   1.41997320e+09,   1.41996960e+09, ...,
         1.40066280e+09,   1.40065920e+09,   1.40065560e+09])

## blending
most work in here is put the to model together

In [134]:
HRRR_1_path = "Fanova_input/GHI_HRRRonly_Site1.csv"
HRRR_1 = pd.read_csv(data_path+HRRR_1_path)
print(HRRR_1.shape)

HRRR_2_path = "Fanova_input/GHI_HRRRonly_Site2.csv"
HRRR_2 = pd.read_csv(data_path+HRRR_2_path)
print(HRRR_2.shape)

HRRR_3_path = "Fanova_input/GHI_HRRRonly_Site3.csv"
HRRR_3 = pd.read_csv(data_path+HRRR_3_path)
print(HRRR_3.shape)

HRRR_4_path = "Fanova_input/GHI_HRRRonly_Site4.csv"
HRRR_4 = pd.read_csv(data_path+HRRR_4_path)
print(HRRR_4.shape)

HRRR_5_path = "Fanova_input/GHI_HRRRonly_Site5.csv"
HRRR_5 = pd.read_csv(data_path+HRRR_5_path)
print(HRRR_5.shape)

HRRR_6_path = "Fanova_input/GHI_HRRRonly_Site6.csv"
HRRR_6 = pd.read_csv(data_path+HRRR_6_path)
print(HRRR_6.shape)

HRRR_7_path = "Fanova_input/GHI_HRRRonly_Site7.csv"
HRRR_7 = pd.read_csv(data_path+HRRR_7_path)
print(HRRR_7.shape)


(3144, 25)
(3153, 25)
(3018, 25)
(2867, 25)
(2881, 25)
(2941, 25)
(2926, 25)


In [137]:
intertime_1 = np.intersect1d(NAM_1['time'], HRRR_1['time'])
print(intertime_1.shape)

intertime_2 = np.intersect1d(NAM_2['time'], HRRR_2['time'])
print(intertime_2.shape)

intertime_3 = np.intersect1d(NAM_3['time'], HRRR_3['time'])
print(intertime_3.shape)

intertime_4 = np.intersect1d(NAM_4['time'], HRRR_4['time'])
print(intertime_4.shape)

intertime_5 = np.intersect1d(NAM_5['time'], HRRR_5['time'])
print(intertime_5.shape)

intertime_6 = np.intersect1d(NAM_6['time'], HRRR_6['time'])
print(intertime_6.shape)

intertime_7 = np.intersect1d(NAM_7['time'], HRRR_7['time'])
print(intertime_7.shape)


(2256,)
(1647,)
(905,)
(1649,)
(1796,)
(2233,)
(1868,)


In [140]:
HRRR_1 = np.array(HRRR_1)
HRRR_2 = np.array(HRRR_2)
HRRR_3 = np.array(HRRR_3)
HRRR_4 = np.array(HRRR_4)
HRRR_5 = np.array(HRRR_5)
HRRR_6 = np.array(HRRR_6)
HRRR_7 = np.array(HRRR_7)
NAM_1 = np.array(NAM_1)
NAM_2 = np.array(NAM_2)
NAM_3 = np.array(NAM_3)
NAM_4 = np.array(NAM_4)
NAM_5 = np.array(NAM_5)
NAM_6 = np.array(NAM_6)
NAM_7 = np.array(NAM_7)

In [177]:
def pick(a,b):
    mask = np.zeros(a.shape[0],dtype = np.bool)
    for i in range(b.shape[0]):
        mask = mask+(a[:,0]==b[i])
    return a[mask]

HRRR_1i = pick(HRRR_1,intertime_1)
print(HRRR_1i.shape)
HRRR_2i = pick(HRRR_2,intertime_2)
print(HRRR_2i.shape)
HRRR_3i = pick(HRRR_3,intertime_3)
print(HRRR_3i.shape)
HRRR_4i = pick(HRRR_4,intertime_4)
print(HRRR_4i.shape)
HRRR_5i = pick(HRRR_5,intertime_5)
print(HRRR_5i.shape)
HRRR_6i = pick(HRRR_6,intertime_6)
print(HRRR_6i.shape)
HRRR_7i = pick(HRRR_7,intertime_7)
print(HRRR_7i.shape)

NAM_1i = pick(NAM_1,intertime_1)
print(NAM_1i.shape)
NAM_2i = pick(NAM_2,intertime_2)
print(NAM_2i.shape)
NAM_3i = pick(NAM_3,intertime_3)
print(NAM_3i.shape)
NAM_4i = pick(NAM_4,intertime_4)
print(NAM_4i.shape)
NAM_5i = pick(NAM_5,intertime_5)
print(NAM_5i.shape)
NAM_6i = pick(NAM_6,intertime_6)
print(NAM_6i.shape)
NAM_7i = pick(NAM_7,intertime_7)
print(NAM_7i.shape)

(2256, 25)
(1647, 25)
(905, 25)
(1649, 25)
(1796, 25)
(2233, 25)
(1868, 25)
(2256, 28)
(1647, 28)
(905, 28)
(1649, 28)
(1796, 28)
(2233, 28)
(1868, 28)


In [178]:
NAM_i = NAM_1i
NAM_i = np.concatenate((NAM_i, NAM_2i), axis=0)
NAM_i = np.concatenate((NAM_i, NAM_3i), axis=0)
NAM_i = np.concatenate((NAM_i, NAM_4i), axis=0)
NAM_i = np.concatenate((NAM_i, NAM_5i), axis=0)
NAM_i = np.concatenate((NAM_i, NAM_6i), axis=0)
NAM_i = np.concatenate((NAM_i, NAM_7i), axis=0)
print(NAM_i.shape)

HRRR_i = HRRR_1i
HRRR_i = np.concatenate((HRRR_i, HRRR_2i), axis=0)
HRRR_i = np.concatenate((HRRR_i, HRRR_3i), axis=0)
HRRR_i = np.concatenate((HRRR_i, HRRR_4i), axis=0)
HRRR_i = np.concatenate((HRRR_i, HRRR_5i), axis=0)
HRRR_i = np.concatenate((HRRR_i, HRRR_6i), axis=0)
HRRR_i = np.concatenate((HRRR_i, HRRR_7i), axis=0)
print(HRRR_i.shape)

NAM_HRRR_i = np.concatenate((NAM_i, HRRR_i), axis=1)
print(NAM_HRRR_i.shape)

(12354, 28)
(12354, 25)
(12354, 53)


In [207]:
print(NAM_HRRR_i[0])
train_size = 12354-1868
valid_size = 1000
test_size = 868

# train_size = 1700
# valid_size = 500
# test_size = 868

mask = np.ones(NAM_HRRR_i.shape[1],dtype = np.bool)
mask[1] = 0
mask[28] = 0
mask[29] = 0
NAM_HRRR_i_label = NAM_HRRR_i[:,1]
NAM_HRRR_i_data = NAM_HRRR_i[:,mask]
print(NAM_HRRR_ii.shape)

NAM_HRRR_i_train_data=NAM_HRRR_i_data[:train_size]
NAM_HRRR_i_valid_data=NAM_HRRR_i_data[train_size:train_size+valid_size]
NAM_HRRR_i_test_data=NAM_HRRR_i_data[train_size+valid_size:train_size+valid_size+test_size]
print(NAM_HRRR_i_train_data.shape)
print(NAM_HRRR_i_valid_data.shape)
print(NAM_HRRR_i_test_data.shape)
NAM_HRRR_i_train_label=NAM_HRRR_i_label[:train_size]
NAM_HRRR_i_valid_label=NAM_HRRR_i_label[train_size:train_size+valid_size]
NAM_HRRR_i_test_label=NAM_HRRR_i_label[train_size+valid_size:train_size+valid_size+test_size]
print(NAM_HRRR_i_train_label.shape)
print(NAM_HRRR_i_valid_label.shape)
print(NAM_HRRR_i_test_label.shape)

[  1.40934600e+09   5.40391663e+02   3.37509308e+02   0.00000000e+00
   2.31300003e+02   8.47470015e-02   7.26900017e-03   3.02657104e+02
  -7.55980015e-01   4.66315603e+00   9.87330078e+04   7.44163284e+01
   1.93750000e+01   0.00000000e+00   0.00000000e+00   1.00000000e+02
   9.70492477e+01   9.77807312e+01   9.33858032e+01   1.01982881e+04
   1.66152893e+03   6.34460022e+02   8.70124451e+02   9.18799973e+01
   5.14440384e+01   2.46143753e+02   4.00500000e+01  -8.83700000e+01
   1.40934600e+09   5.40391663e+02   6.07083000e+02   7.68254456e+02
   0.00000000e+00   8.23002000e-04   0.00000000e+00   3.04175000e+02
  -1.98077000e-01   5.20866000e+00   9.87576000e+04   5.40902000e+01
   0.00000000e+00   0.00000000e+00   3.00001000e+00   0.00000000e+00
   0.00000000e+00   3.00001000e+00   7.63023000e+01   5.28934000e+01
   6.34460022e+02   8.70124451e+02   9.18799973e+01   5.14440384e+01
   2.46143753e+02]
(12354, 50)
(10486, 50)
(1000, 50)
(868, 50)
(10486,)
(1000,)
(868,)


In [204]:
NAM_HRRR_i_train_data = NAM_HRRR_i_train_data.reshape((NAM_HRRR_i_train_data.shape[0],1,NAM_HRRR_i_train_data.shape[1]))
NAM_HRRR_i_valid_nam = NAM_HRRR_i_valid_data[:,1]
NAM_HRRR_i_valid_data = NAM_HRRR_i_valid_data.reshape((NAM_HRRR_i_valid_data.shape[0],1,NAM_HRRR_i_valid_data.shape[1]))

In [216]:
## training Dense
nn = Sequential()
#nn.add()
nn.add(BatchNormalization(input_shape = NAM_HRRR_i_train_data.shape[1:]))
#ls.add(LSTM(100, input_shape = NAM_HRRR_i_train_data.shape))
nn.add(Dense(500,activation = 'relu'))
nn.add(Dense(200,activation = 'relu'))
nn.add(Dense(200, activation = 'relu'))
nn.add(Dense(1, activation = 'relu'))
#ls.compile(optimizer='adam', loss='mae')
# sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
nn.compile(optimizer = 'rmsprop', loss = 'mean_squared_error')
nn.fit(NAM_HRRR_i_train_data, NAM_HRRR_i_train_label,shuffle=True,epochs = 1000, batch_size = 48)

Epoch 1/1000
10486/10486 [==============================] - 3s 249us/step - loss: 25563.9570
Epoch 2/1000
10486/10486 [==============================] - 1s 85us/step - loss: 18491.1032
Epoch 3/1000
10486/10486 [==============================] - 1s 95us/step - loss: 17684.7770
Epoch 4/1000
10486/10486 [==============================] - 1s 90us/step - loss: 17011.8850
Epoch 5/1000
10486/10486 [==============================] - 1s 86us/step - loss: 17039.7393
Epoch 6/1000
10486/10486 [==============================] - 1s 86us/step - loss: 16665.6050
Epoch 7/1000
10486/10486 [==============================] - 1s 89us/step - loss: 16487.0695
Epoch 8/1000
10486/10486 [==============================] - 1s 86us/step - loss: 16231.8575
Epoch 9/1000
10486/10486 [==============================] - 1s 87us/step - loss: 16202.7779
Epoch 10/1000
10486/10486 [==============================] - 1s 88us/step - loss: 15996.4676
Epoch 11/1000
10486/10486 [==============================] - 1s 87us/step - lo

10486/10486 [==============================] - 1s 82us/step - loss: 6178.1367
Epoch 90/1000
10486/10486 [==============================] - 1s 82us/step - loss: 6259.6800
Epoch 91/1000
10486/10486 [==============================] - 1s 80us/step - loss: 6243.9744
Epoch 92/1000
10486/10486 [==============================] - 1s 80us/step - loss: 6147.9991
Epoch 93/1000
10486/10486 [==============================] - 1s 80us/step - loss: 6211.9077
Epoch 94/1000
10486/10486 [==============================] - 1s 81us/step - loss: 6208.3322
Epoch 95/1000
10486/10486 [==============================] - 1s 82us/step - loss: 6087.2757
Epoch 96/1000
10486/10486 [==============================] - 1s 80us/step - loss: 5907.4477
Epoch 97/1000
10486/10486 [==============================] - 1s 80us/step - loss: 5965.1266
Epoch 98/1000
10486/10486 [==============================] - 1s 82us/step - loss: 5868.8369
Epoch 99/1000
10486/10486 [==============================] - 1s 82us/step - loss: 6122.1218
Ep

10486/10486 [==============================] - 1s 84us/step - loss: 3432.9871
Epoch 178/1000
10486/10486 [==============================] - 1s 85us/step - loss: 3424.0313
Epoch 179/1000
10486/10486 [==============================] - 1s 85us/step - loss: 3347.2705
Epoch 180/1000
10486/10486 [==============================] - 1s 85us/step - loss: 3585.1015
Epoch 181/1000
10486/10486 [==============================] - 1s 80us/step - loss: 3409.9855
Epoch 182/1000
10486/10486 [==============================] - 1s 83us/step - loss: 3388.1021
Epoch 183/1000
10486/10486 [==============================] - 1s 80us/step - loss: 3448.7267
Epoch 184/1000
10486/10486 [==============================] - 1s 79us/step - loss: 3472.8277
Epoch 185/1000
10486/10486 [==============================] - 1s 79us/step - loss: 3455.1269
Epoch 186/1000
10486/10486 [==============================] - 1s 83us/step - loss: 3274.6496
Epoch 187/1000
10486/10486 [==============================] - 1s 84us/step - loss: 33

10486/10486 [==============================] - 1s 82us/step - loss: 2521.5269
Epoch 266/1000
10486/10486 [==============================] - 1s 87us/step - loss: 2530.1444
Epoch 267/1000
10486/10486 [==============================] - 1s 87us/step - loss: 2518.8047
Epoch 268/1000
10486/10486 [==============================] - 1s 84us/step - loss: 2518.0559
Epoch 269/1000
10486/10486 [==============================] - 1s 82us/step - loss: 2404.7296
Epoch 270/1000
10486/10486 [==============================] - 1s 82us/step - loss: 2345.2621
Epoch 271/1000
10486/10486 [==============================] - 1s 83us/step - loss: 2408.8029
Epoch 272/1000
10486/10486 [==============================] - 1s 88us/step - loss: 2395.0384
Epoch 273/1000
10486/10486 [==============================] - 1s 82us/step - loss: 2391.8089
Epoch 274/1000
10486/10486 [==============================] - 1s 83us/step - loss: 2400.7996
Epoch 275/1000
10486/10486 [==============================] - 1s 86us/step - loss: 22

10486/10486 [==============================] - 1s 82us/step - loss: 2042.9190
Epoch 354/1000
10486/10486 [==============================] - 1s 82us/step - loss: 1931.2075
Epoch 355/1000
10486/10486 [==============================] - 1s 82us/step - loss: 1990.5372
Epoch 356/1000
10486/10486 [==============================] - 1s 82us/step - loss: 1912.5913
Epoch 357/1000
10486/10486 [==============================] - 1s 82us/step - loss: 1994.3551
Epoch 358/1000
10486/10486 [==============================] - 1s 81us/step - loss: 1989.5217
Epoch 359/1000
10486/10486 [==============================] - 1s 86us/step - loss: 1979.1221
Epoch 360/1000
10486/10486 [==============================] - 1s 81us/step - loss: 2008.2420
Epoch 361/1000
10486/10486 [==============================] - 1s 82us/step - loss: 1890.9980
Epoch 362/1000
10486/10486 [==============================] - 1s 81us/step - loss: 2043.3413
Epoch 363/1000
10486/10486 [==============================] - 1s 84us/step - loss: 18

...

10486/10486 [==============================] - 1s 82us/step - loss: 1371.7586
Epoch 617/1000
10486/10486 [==============================] - 1s 80us/step - loss: 1368.5767
Epoch 618/1000
10486/10486 [==============================] - 1s 81us/step - loss: 1393.5599
Epoch 619/1000
10486/10486 [==============================] - 1s 81us/step - loss: 1265.0865
Epoch 620/1000
10486/10486 [==============================] - 1s 82us/step - loss: 1259.4652
Epoch 621/1000
10486/10486 [==============================] - 1s 82us/step - loss: 1318.1213
Epoch 622/1000
10486/10486 [==============================] - 1s 84us/step - loss: 1251.9393
Epoch 623/1000
10486/10486 [==============================] - 1s 81us/step - loss: 1322.0791
Epoch 624/1000
10486/10486 [==============================] - 1s 81us/step - loss: 1212.5589
Epoch 625/1000
10486/10486 [==============================] - 1s 85us/step - loss: 1256.5075
Epoch 626/1000
10486/10486 [==============================] - 1s 81us/step - loss: 12

10486/10486 [==============================] - 1s 83us/step - loss: 1177.0975
Epoch 705/1000
10486/10486 [==============================] - 1s 81us/step - loss: 1181.6943
Epoch 706/1000
10486/10486 [==============================] - 1s 81us/step - loss: 1221.1247
Epoch 707/1000
10486/10486 [==============================] - 1s 81us/step - loss: 1159.7474
Epoch 708/1000
10486/10486 [==============================] - 1s 81us/step - loss: 1157.6309
Epoch 709/1000
10486/10486 [==============================] - 1s 82us/step - loss: 1223.8806
Epoch 710/1000
10486/10486 [==============================] - 1s 82us/step - loss: 1198.8390
Epoch 711/1000
10486/10486 [==============================] - 1s 81us/step - loss: 1114.0077
Epoch 712/1000
10486/10486 [==============================] - 1s 82us/step - loss: 1178.1744
Epoch 713/1000
10486/10486 [==============================] - 1s 81us/step - loss: 1091.9980
Epoch 714/1000
10486/10486 [==============================] - 1s 81us/step - loss: 10

10486/10486 [==============================] - 1s 88us/step - loss: 1080.4513
Epoch 793/1000
10486/10486 [==============================] - 1s 86us/step - loss: 1038.2612
Epoch 794/1000
10486/10486 [==============================] - 1s 85us/step - loss: 1110.3002
Epoch 795/1000
10486/10486 [==============================] - 1s 89us/step - loss: 1093.7957
Epoch 796/1000
10486/10486 [==============================] - 1s 113us/step - loss: 1035.5687
Epoch 797/1000
10486/10486 [==============================] - 1s 125us/step - loss: 1080.7832
Epoch 798/1000
10486/10486 [==============================] - 1s 109us/step - loss: 1117.2594
Epoch 799/1000
10486/10486 [==============================] - 1s 108us/step - loss: 1009.5909
Epoch 800/1000
10486/10486 [==============================] - 1s 121us/step - loss: 1099.7855
Epoch 801/1000
10486/10486 [==============================] - 1s 109us/step - loss: 1053.2628
Epoch 802/1000
10486/10486 [==============================] - 1s 137us/step - l

10486/10486 [==============================] - 1s 86us/step - loss: 1031.5170
Epoch 880/1000
10486/10486 [==============================] - 1s 86us/step - loss: 976.4777
Epoch 881/1000
10486/10486 [==============================] - 1s 82us/step - loss: 1041.0232
Epoch 882/1000
10486/10486 [==============================] - 1s 82us/step - loss: 1002.4562
Epoch 883/1000
10486/10486 [==============================] - 1s 81us/step - loss: 983.8418
Epoch 884/1000
10486/10486 [==============================] - 1s 81us/step - loss: 962.9579
Epoch 885/1000
10486/10486 [==============================] - 1s 82us/step - loss: 1033.6270
Epoch 886/1000
10486/10486 [==============================] - 1s 84us/step - loss: 1025.6106
Epoch 887/1000
10486/10486 [==============================] - 1s 83us/step - loss: 960.2505
Epoch 888/1000
10486/10486 [==============================] - 1s 82us/step - loss: 1026.4750
Epoch 889/1000
10486/10486 [==============================] - 1s 81us/step - loss: 980.28

10486/10486 [==============================] - 1s 81us/step - loss: 955.5634
Epoch 968/1000
10486/10486 [==============================] - 1s 80us/step - loss: 1020.0017
Epoch 969/1000
10486/10486 [==============================] - 1s 82us/step - loss: 917.3409
Epoch 970/1000
10486/10486 [==============================] - 1s 89us/step - loss: 915.2747
Epoch 971/1000
10486/10486 [==============================] - 1s 94us/step - loss: 934.0864
Epoch 972/1000
10486/10486 [==============================] - 1s 107us/step - loss: 942.8118
Epoch 973/1000
10486/10486 [==============================] - 1s 94us/step - loss: 978.5614
Epoch 974/1000
10486/10486 [==============================] - 1s 87us/step - loss: 867.7664
Epoch 975/1000
10486/10486 [==============================] - 1s 96us/step - loss: 950.6835:
Epoch 976/1000
10486/10486 [==============================] - 1s 85us/step - loss: 881.4710
Epoch 977/1000
10486/10486 [==============================] - 1s 85us/step - loss: 955.0673


In [217]:
pred = (nn.predict(NAM_HRRR_i_valid_data))
print(np.linalg.norm(NAM_HRRR_i_valid_label-pred.reshape(pred.shape[0]),2))
print(np.linalg.norm(NAM_HRRR_i_valid_label-NAM_HRRR_i_valid_nam,2))

5713.76135396
6955.21702679


In [213]:
# print(pred)
# rand_nam_1_valid_label = (rand_nam_1_valid_label+np.abs(rand_nam_1_valid_label))/2.


# visualize results
plt.clf()
fig.clear()
fig, ax = plt.subplots()
fig.set_size_inches(500, 3)

ax.plot(NAM_HRRR_i_valid_data[:,0],NAM_HRRR_i_valid_label.reshape(NAM_HRRR_i_valid_label.shape[0]),alpha = 0.5, color = "k")
ax.plot(NAM_HRRR_i_valid_data[:,0],NAM_HRRR_i_valid_nam,alpha = 0.5, color = "y")
ax.plot(NAM_HRRR_i_valid_data[:,0],pred,alpha = 0.5, color = "g")

#plt.show()
plt.savefig("model_4.pdf")

/usr/local/lib/python3.6/site-packages/matplotlib/pyplot.py:523: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


## LSTM
create data set for LSTM

In [232]:
def convert_LSTM(data, label, look_back = 1):
    look_back = look_back-1
    LSTM_data = []
    LSTM_label = []
    for i in range(data.shape[0]-look_back):
        temp_data = data[i:(i+look_back)+1]
        LSTM_data.append(temp_data)
        LSTM_label.append(label[i+look_back])
    return np.array(LSTM_data), np.array(LSTM_label)

sample_data = np.array([[1,2],[4,2],[3,5],[2,5],[7,2],[2,6],[5,1],[2,4]])
sample_label = np.array([6,5,4,3,2,1,2,3])
ldata,llabel = convert_LSTM(sample_data, sample_label, look_back = 3)
print(ldata,llabel)

[[[1 2]
  [4 2]
  [3 5]]

 [[4 2]
  [3 5]
  [2 5]]

 [[3 5]
  [2 5]
  [7 2]]

 [[2 5]
  [7 2]
  [2 6]]

 [[7 2]
  [2 6]
  [5 1]]

 [[2 6]
  [5 1]
  [2 4]]] [4 3 2 1 2 3]
